In [122]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [92]:
problem = '220D69802BE00A0803711E1441B1006E39C318A12730C200DCE66D2CCE360FA0055652CD32966E3004677EDF600B0803B1361741510076254138D8A00E4FFF3E3393ABE4FC7AC10410010799D2A4430003764DBE281802F3102CA00D4840198430EE0E00021D04E3F41F84AE0154DFDE65A17CCBFAFA14ADA56854FE5E3FD5BCC53B0D2598027A00848C63F2B918C7E513DEC3290051B3867E009CCC5FE46BD520007FE5E8AD344B37583D0803E40085475887144C01A8C10FE2B9803B0720D45A3004652FD8FA05F80122CAF91E5F50E66BEF8AB000BB0F4802039C20917B920B9221200ABF0017B9C92CCDC76BD3A8C4012CCB13CB22CDB243E9C3D2002067440400D9BE62DAC4D2DC0249BF76B6F72BE459B279F759AE7BE42E0058801CC059B08018A0070012CEC045BA01006C03A8000D46C02FA000A8EA007200800E00618018E00410034220061801D36BF178C01796FC52B4017100763547E86000084C7E8910AC0027E9B029FE2F4952F96D81B34C8400C24AA8CDAF4F1E98027C00FACDE3BA86982570D13AA640195CD67B046F004662711E989C468C01F1007A10C4C8320008742287117C401A8C715A3FC2C8EB3777540048272DFE7DE1C0149AC8BC9E79D63200B674013978E8BE5E3A2E9AA3CCDD538C01193CFAB0A146006AA00087C3E88B130401D8E304A239802F39FAC922C0169EA3248DF2D600247C89BCDFE9CA7FFD8BB49686236C9FF9795D80C0139BEC4D6C017978CF78C5EB981FCE7D4D801FA9FB63B14789534584010B5802F3467346D2C1D1E080355B00424FC99290C7E5D729586504803A2D005E677F868C271AA479CEEB131592EE5450043A932697E6A92C6E164991EFC4268F25A294600B5002A3393B31CC834B972804D2F3A4FD72B928E59219C9C771EC3DC89D1802135C9806802729694A6E723FD6134C0129A019E600'

In [3]:
pt1_test = {
    '8A004A801A8002F478':16,
    '620080001611562C8802118E34':12,
    'C0015000016115A2E0802F182340':23,
    'A0016C880162017C3686B18A3D4780':31,
}

# Part 2 instructions
- Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
- Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
- Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.

In [117]:
from copy import deepcopy
class Packet:
    
    def __init__(self, bit_string):
        self.version, self.type, bit_string = self._read_version_type(bit_string)
        self.value = 0
        
        bit_string, self.child_packets = self._evalute_packet(bit_string)
        #print(self.version)
        self.remaining_string = bit_string
        self.value = self.calculate_value()
        
    def calculate_value(self):
        ptype = self.type
        children = self.child_packets
        cur_val = self.value
        if ptype ==0:
            return sum([ child.value for child in children ])
        if ptype == 1:
            val = 1
            for child in children:
                val *=child.value 
            return val
        if ptype == 2 :
            return min([ child.value for child in children ])
        if ptype == 3 :
            return max([ child.value for child in children ])
        if ptype == 4:
            return cur_val
        if ptype == 5:
            return int(children[0].value > children[1].value)
        if ptype == 6:
            return int(children[0].value < children[1].value)
        if ptype == 7:
            return int(children[0].value == children[1].value)
        
    
    def calculate_version_sum(self):
        if len(self.child_packets) > 0:
            return self.version + sum([ child.calculate_version_sum() for child in self.child_packets])
        else:
            return self.version
        
    def _evalute_packet(self, bit_string):
        children =[]
        if self.type ==4:
            self.value, bit_string = self._read_literal_val(bit_string)
        else:
            length_type_id = bit_string[0]
            bit_string = bit_string[1:]
            
            if length_type_id == '0':
                bit_string, children = self._lt_0_read(bit_string)
            elif length_type_id == '1':
                bit_string, children = self._lt_1_read(bit_string)
            
        return bit_string, children
    
    def _read_version_type(self, bit_string):
        version = int(bit_string[:3],2)
        ptype = int(bit_string[3:6],2)
        
        return version, ptype, bit_string[6:]
    
    def _read_literal_val(self, bit_string):
        read = bit_string[0]
        val_string = bit_string[1:5]
        idx=5
        while read == '1':
            read = bit_string[idx]
            val_string += bit_string[idx+1:idx+5]
            idx+=5
        return int(val_string, 2), bit_string[idx:]
    
  
    def _lt_0_read(self, bit_string):
        length_bits = int(bit_string[:15],2)
        child_string = bit_string[15:15+length_bits]
        children=[]
        while len(child_string) > 0:
            child = Packet(child_string)
            children.append(child)
            child_string = child.remaining_string
            
        return bit_string[15+length_bits:], children
    
    def _lt_1_read(self, bit_string):
        nsubpackets = int(bit_string[:11],2)
        bit_string = bit_string[11:]
        children=[]
        for _ in range(nsubpackets):
            child = Packet(bit_string)
            bit_string = child.remaining_string
            children.append(child)
            
        return children[-1].remaining_string, children
            
    
class HexMessage:
    hex_to_bin={'0' : '0000',
                '1' : '0001',
                '2' : '0010',
                '3' : '0011',
                '4' : '0100',
                '5' : '0101',
                '6' : '0110',
                '7' : '0111',
                '8' : '1000',
                '9' : '1001',
                'A' : '1010',
                'B' : '1011',
                'C' : '1100',
                'D' : '1101',
                'E' : '1110',
                'F' : '1111',
                }
    
    def __init__(self, hex_data):
        self.hexstring = hex_data
        self._bit_string = self._bit_representation()
        self.packet = Packet(self._bit_string)
        
    def return_part_one(self):
        return self.packet.calculate_version_sum()
    
    def return_part_two(self):
        return self.packet.value
        
    def _bit_representation(self):
        bit_string=''
        for l in self.hexstring:
            bit_string+=self.hex_to_bin[l]
        return bit_string

In [118]:
# Test part 1
for string, expected in pt1_test.items():
    test = HexMessage(string)
    print(test.return_part_one() == expected)
    
    

True
True
True
True


In [119]:
test.packet.value

54

In [120]:
prd = HexMessage(problem)

In [123]:
prd.return_part_one()
prd.return_part_two()

1002

1673210814091